In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
import re
from math import log
from collections import OrderedDict
import nltk
import html
import unicodedata
from textblob import TextBlob
import gensim
import nltk
from collections import Counter

In [2]:
df_cop = pd.read_csv('/Users/albert/Desktop/CMU/Internship/GAI/UNGC_COP/COP_NLP/phase_1/PHASE_1_multi/output/decoded output for 51-100/2018_COP_phase_1_result.csv')

In [3]:
def clean_text(text):
    
    cidcompile1=re.compile(r'\(*\s*\(*\s*[cC]\s*i\s*\)*\s*d\s*\:*\s*\:*(?:\(*\s*[cC]\s*i\s*d\s*\:*)?\:*\s*[0-9]+\s*[0-9]{0,}\s*\)*\s*\)*')
    cidcompile2=re.compile(r'\(\s*[cC]\s*i\s*\)*\s*d\s*\:')
    cidcompile3=re.compile(r'\:\s*[0-9]+\s*\)')

    t_str=re.sub(cidcompile1,' ',text)
    t_str=re.sub(cidcompile2,' ',t_str)
    t_str=re.sub(cidcompile3,' ',t_str)
    
    control = re.compile('\x00|\x01|\x02|\x03|\x04|\x05|\x06|\x07|\x08|\x09|\x0a|\x0b|\x0c|\x0d|\x0e|\x0f|\x10|\x11|\x12|\x13|\x14|\x15|\x16|\x17|\x18|\x19|\x1a|\x1b|\x1c|\x1d|\x1e|\x1f|\x7f|\xc2\x80|\xc2\x81|\xc2\x82|\xc2\x83|\xc2\x84|\xc2\x85|\xc2\x86|\xc2\x87|\xc2\x88|\xc2\x89|\xc2\x8a|\xc2\x8b|\xc2\x8c|\xc2\x8d|\xc2\x8e|\xc2\x8f|\xc2\x90|\xc2\x91|\xc2\x92|\xc2\x93|\xc2\x94|\xc2\x95|\xc2\x96|\xc2\x97|\xc2\x98|\xc2\x99|\xc2\x9a|\xc2\x9b|\xc2\x9c|\xc2\x9d|\xc2\x9e|\xc2\x9f')
    
    t_str = re.sub(control,' ',t_str)
    t_str = unicodedata.normalize("NFKD",t_str)
    t_str = re.sub('[\uE000-\uF8B6\uF8C1-\uF8E4]+',' ',t_str)
    t_str = html.unescape(t_str)
    
    return t_str

In [4]:
df_cop['text'] = df_cop['text'].apply(clean_text)
df_cop['translation'] = df_cop['translation'].apply(clean_text)

In [5]:
df_cop.drop(labels=['Unnamed: 0'], inplace=True, axis=1)

In [6]:
df_kw = pd.read_excel('REF - UNGC Taxonomy -2018.xlsx')

In [7]:
df_kw.dropna(how='all', inplace=True)
df_kw['Unnamed: 1'][1:4] = 'Environment'
df_kw['Unnamed: 1'][5:15] = 'Society'
df_kw['Unnamed: 1'][16:23] = 'Economy'
df_kw.drop(54, inplace=True)
df_kw.rename(columns={"Unnamed: 1": "Topic", "Unnamed: 2": "Sub_Category"}, inplace=True)
df_kw.set_index(['Sub_Category'], inplace=True)
df_kw.drop(columns=['Unnamed: {}'.format(i) for i in range(4, 15)], inplace=True)
df_kw.drop(columns=['Unnamed: 0'], inplace=True)
df_kw.rename(columns={"Unnamed: 3": "Key_Words"}, inplace=True)
df_kw.dropna(inplace=True)
#Drop topic column
df_kw.drop(columns='Topic', inplace=True)
df_kw['Key_Words'] = df_kw['Key_Words'].apply(lambda x: x.split(", "))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [8]:
dict_cat_kw = {sub_category: keyword for (sub_category, keyword) in zip(df_kw.index.tolist(), sum(df_kw.values.tolist(), []))}
dict_cat_kw['Climate'] = dict_cat_kw.pop('Climate ')
dict_cat_kw['Water'][dict_cat_kw['Water'].index('ocean,')] = 'ocean'
dict_cat_kw['Forced Labour'][dict_cat_kw['Forced Labour'].index(' involuntary labor')] = 'involuntary labor'
dict_cat_kw['Forced Labour'][dict_cat_kw['Forced Labour'].index('involuntary labour,')] = 'involuntary labour'
dict_cat_kw['Anti-Corruption'][dict_cat_kw['Anti-Corruption'].index('extortion,')] = 'extortion'
dict_cat_kw['Corp. Sustainability Management'][dict_cat_kw['Corp. Sustainability Management'].index('business strategy ')] = 'business strategy'
for cat in dict_cat_kw.keys():
    dict_cat_kw[cat] = [kw.lower() for kw in list(filter(lambda kw: kw != '', dict_cat_kw[cat]))]

In [9]:
df_cop['text'] = df_cop['text'].apply(lambda x: x.lower())
df_cop['translation'] = df_cop['translation'].apply(lambda x: x.lower())

In [10]:
# test = 'this is a foo bar sentences and i want to ngramize it'
# n = 2
# sixgrams = nltk.ngrams(test.split(), n)
# for grams in sixgrams:
#     print(grams)
# kws_all = sum(dict_cat_kw.values(), [])
# kws_all[kws_all.index('ocean,')] = 'ocean'
# kws_all = list(filter(lambda kw: kw != '', kws_all))
# kws_all[kws_all.index(' involuntary labor')] = 'involuntary labor'
# kws_all[kws_all.index('involuntary labour,')] = 'involuntary labour'
# kws_all[kws_all.index('extortion,')] = 'extortion'
# kws_all[kws_all.index('business strategy ')] = 'business strategy'
# kws_all = [kw.lower() for kw in kws_all]
# kws_one = [kw for kw in kws_all if len(kw.split()) == 1]
# kws_two = [kw for kw in kws_all if len(kw.split()) == 2]
# kws_three = [kw for kw in kws_all if len(kw.split()) == 3]
# kws_four = [kw for kw in kws_all if len(kw.split()) == 4]
# kws_five = [kw for kw in kws_all if len(kw.split()) == 5]

In [11]:
def get_subcat(keyword):
    subcat_list = []
    for subcat, kw_list in dict_cat_kw.items():
        if keyword in kw_list:
            subcat_list.append(subcat)
    return subcat_list

In [12]:
kws_all = sum(dict_cat_kw.values(), [])
# kws_all[kws_all.index('ocean,')] = 'ocean'
# kws_all = list(filter(lambda kw: kw != '', kws_all))
# kws_all[kws_all.index(' involuntary labor')] = 'involuntary labor'
# kws_all[kws_all.index('involuntary labour,')] = 'involuntary labour'
# kws_all[kws_all.index('extortion,')] = 'extortion'
# kws_all[kws_all.index('business strategy ')] = 'business strategy'
# kws_all = [kw.lower() for kw in kws_all]
kws_one = [kw for kw in kws_all if len(kw.split()) == 1]
kws_multi = [kw for kw in kws_all if len(kw.split()) > 1]
# kws_two = [kw for kw in kws_all if len(kw.split()) == 2]
# kws_three = [kw for kw in kws_all if len(kw.split()) == 3]
# kws_four = [kw for kw in kws_all if len(kw.split()) == 4]
# kws_five = [kw for kw in kws_all if len(kw.split()) == 5]
corp_id_list = df_cop['corp_id'].tolist()
# subcat_list = dict_cat_kw.keys()
for index, row in df_cop.iterrows():
    txt = ''
    if row['translation'] == 'already in english':
        txt = row['text']
    else:
        txt = row['translation']
    #     txt_words_list = [str(wd) for wd in TextBlob(txt).words]
    #     kw_freq = {kw: txt.count(kw) for kw in kws_all if txt.count(kw) > 0}
    tb = TextBlob(txt)
    word_freq = tb.word_counts
    kw_freq = {kw: word_freq[kw] for kw in kws_one if kw in set(word_freq)}
    for kw in kws_multi:
        freq = txt.count(kw)
        if  freq > 0:
            kw_freq[kw] = freq
    df_cop.loc[index, 'Keyword_Frequency'] = [kw_freq]
    subcat_freq_dict = {subcat: 0 for subcat in dict_cat_kw.keys()}
    for kw, freq in kw_freq.items():
        kw_subcat_list = get_subcat(kw)
        for subcat in kw_subcat_list:
            subcat_freq_dict[subcat] = subcat_freq_dict[subcat] + freq
    df_cop.loc[index, 'Subcategory_Frequency'] = [subcat_freq_dict]

In [13]:
senti_analy_dict = {}
for index, row in df_cop.iterrows():
    content = ''
    if row['translation'] == "already in english":
        content = row['text'].lower()
    else:
        content = row['translation'].lower()
    corp_id = row['corp_id']
    sentence_list = nltk.sent_tokenize(content)
    content_dict = {}
    for sentence in sentence_list:
        word_list = nltk.word_tokenize(sentence)
        word_freq_dict = dict(nltk.FreqDist(word_list))
        words = [word for word in word_freq_dict.keys()]
        for word in words:
            if word not in kws_one:
                del word_freq_dict[word]
        kw_freq_dict = word_freq_dict
        for kw in kws_multi:
            count = sentence.count(kw)
            if count > 0:
                kw_freq_dict[kw] = count
        sentence_sentiment = TextBlob(sentence).sentiment.polarity
        subcat_sentiment_dict = {subcat: 0 for subcat in dict_cat_kw.keys()}
        for kw, freq in kw_freq_dict.items():
            subcat_list = get_subcat(kw)
            for subcat in subcat_list:
                subcat_sentiment_dict[subcat] += (freq * sentence_sentiment)
        content_dict[sentence] = subcat_sentiment_dict
    content_subcat_senti_dict = {subcat: 0 for subcat in dict_cat_kw.keys()}
    for sentence, subcat_senti_dict in content_dict.items():
        for subcat in subcat_senti_dict.keys():
            content_subcat_senti_dict[subcat] += subcat_senti_dict[subcat]
    senti_analy_dict[corp_id] = content_subcat_senti_dict

In [16]:
senti_analy_dict[456531]

{'Food and Agriculture': 0,
 'Water': 0,
 'Oceans': 0,
 'Human Rights': 0.3206060606060606,
 'Labour Rights': 0.3206060606060606,
 'Children’s Rights': 0,
 'Women’s Empowerment': 0,
 'Gender Equality': 0,
 'Inequality': 0,
 'Poverty': 0,
 'Migrant Workers': 0,
 'Indigenous Peoples': 0,
 'Forced Labour': 0,
 'Health': 0,
 'Anti-Corruption': 0.3206060606060606,
 'Rule of Law': 0,
 'Peace': 0,
 'Supply Chain': 0,
 'Sustainable Finance': 0,
 'Reporting': 0.2554545454545455,
 'Breakthrough Innovation': 0,
 'Corp. Sustainability Management': 0,
 'Climate': 0}

In [17]:
df_cop

,corp_id,file_name,file_size_in_bytes,page_num,text,file_format,translation,Subcategory_Frequency
0,458061,Grace_letter_REPSA_January_30th_2018.pdf,25447,1.0,"a reforestadora de palmas de el peten, s. a. ...",image-based-PDF,already in english,"{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."
1,458291,ETESA_Grace_Letter_2018.pdf,142529,1.0,"etesr unimos panamé con energia january 31,...",image-based-PDF,already in english,"{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."
2,456101,Drei_Elemente_GmbH_Grace_Letter_UN_Global_Comp...,232240,1.0,drei elemente gmbh | lothringer strate 36 |...,image-based-PDF,already in english,"{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."
3,457241,Carta_de_Gracia_PM-DD.pdf,403936,1.0,"cancun, q. roo a 29 de enero del 2018. p...",image-based-PDF,"cancun, q. roo as of january 29, 2018. un...","{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."
4,456531,UNGC_Request_adjusted_reproting_cycle_.pdf,658060,1.0,~ gy) 2018.01.25 h.e. antonio guterres secr...,image-based-PDF,already in english,"{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."
5,457841,Reporting_Cycle_Adjustment.pdf,310472,1.0,to our stakeholders: we recognize that a key ...,image-based-PDF,already in english,"{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."
6,458051,Grace_letter_PAPSA_January_30th_2018.pdf,27311,1.0,"platanera del pacifico, s. a. guatemala, janu...",image-based-PDF,already in english,"{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."
7,455111,Carta_Gracia_Pacto_Mundial_GFGCA.pdf,147126,1.0,fm tm | gran wf | costa adeje hoteles | *****...,image-based-PDF,fm tm ? great wf costa adeje hotels ? ***** ...,"{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."
8,458201,Anudal_Carta_de_gracia.pdf,213308,1.0,a anudal industrial sl badalona 31 de enero ...,image-based-PDF,"audal industrial sl badalona january 31, 201...","{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."
9,454851,cop.pdf,612974,1.0,"mazatlan mazatlan, sinaloa, january 19, ...",image-based-PDF,already in english,"{'Food and Agriculture': 0, 'Water': 0, 'Ocean..."


In [18]:
# In [8]: iterables = [['bar', 'baz', 'foo', 'qux'], ['one', 'two']]

# In [9]: pd.MultiIndex.from_product(iterables, names=['first', 'second'])
# Out[9]: 
# MultiIndex([('bar', 'one'),
#             ('bar', 'two'),
#             ('baz', 'one'),
#             ('baz', 'two'),
#             ('foo', 'one'),
#             ('foo', 'two'),
#             ('qux', 'one'),
#             ('qux', 'two')],
#            names=['first', 'second'])
# corp_id_list = df_cop['corp_id'].tolist()
# subcat_list = dict_cat_kw.keys()
df_result = pd.DataFrame(index=pd.MultiIndex.from_product([df_cop['corp_id'], dict_cat_kw.keys()], 
                                                          names=['Cop_File_ID', 'Sub_Category']), 
                         columns=['Mentioned', 'Level_of_Mention', 'Sentiment_Score'])
df_cop.set_index('corp_id', inplace=True)
for index, row in df_result.iterrows():
    corp_id = index[0]
    subcat = index[1]
    subcat_freq_dict = df_cop.loc[corp_id, 'Subcategory_Frequency']
    if subcat_freq_dict[subcat] >= 5:
        row['Mentioned'] = 'Yes'
        row['Level_of_Mention'] = 'HEAVY'
    elif subcat_freq_dict[subcat] >= 3:
        row['Mentioned'] = 'Yes'
        row['Level_of_Mention'] = 'MEDIAN'
    elif subcat_freq_dict[subcat] > 0:
        row['Mentioned'] = 'No'
        row['Level_of_Mention'] = 'LIGHT'
    elif subcat_freq_dict[subcat] == 0:
        row['Mentioned'] = 'No'
        row['Level_of_Mention'] = 'NO MENTION'
    row['Sentiment_Score'] = senti_analy_dict[corp_id][subcat]

In [21]:
df_result.loc[458251]

,Mentioned,Level_of_Mention,Sentiment_Score
Sub_Category,,,
Food and Agriculture,No,LIGHT,-0.0111111
Water,No,NO MENTION,0
Oceans,No,NO MENTION,0
Human Rights,Yes,HEAVY,1.08068
Labour Rights,Yes,HEAVY,2.61333
Children’s Rights,Yes,MEDIAN,0.225
Women’s Empowerment,No,NO MENTION,0
Gender Equality,No,NO MENTION,0
Inequality,No,NO MENTION,0


In [20]:
df_result.to_csv('/Users/albert/Desktop/CMU/Internship/GAI/UNGC_COP/COP_NLP/phase_3/output/Phase_3_evaluation.csv')